In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from gensim.models import LdaModel, CoherenceModel
from gensim import corpora
from scipy import stats as st

In [2]:
mutations = pd.read_table('../data/all_muts_pass_and_polyphen.txt', sep=' ')
gene_exp = pd.read_table('../data/all_tcga_gene_exp.txt', sep='\t')
maf = pd.read_table('../data/all_tcga_maf.txt', sep=' ')
cin_loc = pd.read_table('../data/cin_locations.txt')
sanchez_vega = pd.read_table('../data/sanchez_vega_pws_1026.csv', sep=';')

/scratch/64055646/ipykernel_8606/4241139062.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations = pd.read_table('../data/all_muts_pass_and_polyphen.txt', sep=' ')


In [3]:
file = pd.read_table('../steps/discretized_9_6bins.features', dtype=str)

In [4]:
lda = LdaModel.load(f'../steps/gensim/lda/lda_t5_f10.model')
df = file.drop(['Sample', 'Chr'], axis = 1)
listedDf = df.values.tolist()
dirichlet_dict = corpora.Dictionary(listedDf)
bow_corpus = [dirichlet_dict.doc2bow(text) for text in listedDf]

In [5]:
def create_heatmap(model, df):
    topics = lda.get_document_topics(bow = bow_corpus, minimum_probability = 0)
    
    new_df = pd.DataFrame(df['Sample'])
    new_df['Chr'] = df['Chr']
    new_df['Chr'] = pd.to_numeric(new_df['Chr'])
    
    columns = [[],[],[],[],[]]
    for topic in topics:
        for w in topic:
            columns[w[0]].append(w[1])
    
    new_df['1'] = columns[0]
    new_df['2'] = columns[1]
    new_df['3'] = columns[2]
    new_df['4'] = columns[3]
    new_df['5'] = columns[4]
    new_df = new_df.sort_values(by='Chr')
    return new_df


In [ ]:
weights_df = create_heatmap(lda, file)

In [ ]:
mut = weights_df.reset_index(drop=True)
mut = mut.rename(columns = {'Sample': 'sampleID'})
mut= mut.drop_duplicates(subset = ["sampleID"])
mut = pd.merge(left = mut, right = maf[['sampleID', 'mutatated_genes']], on='sampleID')

gene_list = ['TP53', 'PTEN', 'NF1', 'MYC', 'CDK12', 'CCNE1', 'BRCA2', 'BRCA1']
# gene_list = list(cin_loc['Gene'])

for gene in gene_list:
    mut[gene] = np.where(mut['mutatated_genes'].str.contains(gene), True, False)


mut

In [ ]:
gene_expT = gene_exp.transpose()
gene_expT['Sample'] = gene_expT.index
gene_expT

In [ ]:
exp = weights_df.reset_index(drop=True).drop(['Chr'], axis = 1).drop_duplicates(subset = ["Sample"])
exp.columns = ['Sample', 'Sig1', 'Sig2', 'Sig3', 'Sig4', 'Sig5']
exp = pd.merge(left = exp, right = gene_expT, on = 'Sample')

In [ ]:
exp.index = exp['Sample']
exp = exp.drop(['Sample'], axis = 1)
exp

In [ ]:
exp_final = exp.drop(['Sig1', 'Sig2', 'Sig3', 'Sig4', 'Sig5'], axis = 1)
exp_final

In [ ]:
correlation_df = pd.DataFrame(columns=exp_final.columns, index=['Signature_1', 'Signature_2', 'Signature_3', 'Signature_4', 'Signature_5'])

for signature in range(1, 6):
    for gene in correlation_df.columns:
        correlation_df[gene][f'Signature_{signature}'] = st.pearsonr(exp[f'Sig{signature}'], exp_final[gene])[0]

In [ ]:
correlation_df

In [ ]:
correlation_df.to_csv("/home/janneae/cns/steps/allgenes.correlation", header=True, index=None, sep='\t')

In [14]:
pval_df = pd.DataFrame(columns=exp_final.columns, index=['Signature_1', 'Signature_2', 'Signature_3', 'Signature_4', 'Signature_5'])

for signature in range(1, 6):
    for gene in pval_df.columns:
        pval_df[gene][f'Signature_{signature}'] = st.pearsonr(exp[f'Sig{signature}'], exp_final[gene])[1]
pval_df

/home/janneae/miniconda3/envs/cns/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,PRAMEF26,OR8K3,CCDC163,PAGR1,RPS4Y2,TMEM265,TEX13D,ADORA3,DUXB,TEX13C
Signature_1,0.012675,0.596771,0.895017,0.0,0.0,0.000047,0.0,0.801595,0.01378,0.001094,...,0.836763,0.47595,0.02764,0.425968,0.000389,0.321777,0.31657,0.00152,0.730535,0.157969
Signature_2,0.000136,0.000018,0.0,0.000008,0.0,0.0,0.005375,0.0,0.68693,0.0,...,0.0,0.000022,0.003073,0.000009,0.000035,0.444398,0.001043,0.0,0.000358,0.0
Signature_3,0.208807,0.000005,0.0,0.001808,0.0,0.0,0.000008,0.93283,0.014726,0.000002,...,0.000184,0.000037,0.000148,0.000036,0.118924,0.164947,0.000974,0.0,0.023596,0.000233
Signature_4,0.000002,0.515798,0.0,0.0,0.0,0.782455,0.0,0.0,0.474494,0.0,...,0.00203,0.031812,0.088751,0.042044,0.0,0.018021,0.11278,0.522934,0.130458,0.000017
Signature_5,0.001759,0.826763,0.000086,0.0,0.0,0.186261,0.000137,0.009384,0.714224,0.000004,...,0.042369,0.86862,0.008071,0.105247,0.867442,0.069017,0.109953,0.659048,0.149203,0.616586
